In [2]:
import numpy as np
import json
import pandas as pd
import os
import re
import ast
import xml.etree.ElementTree as ET
import glob

from __future__ import print_function
from gensim import corpora

from nltk.sem.logic import *
from nltk.sem.logic import LogicParser



##日本語のため##
import MeCab
mecab = MeCab.Tagger('-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd')

In [10]:
#SNLIからテキスト部分取り出し
#ここで文書がユニークかどうかを見ている
#そのため，ペアにしたい場合は注意が必要

lines = []
with open('') as f:
    line = f.readline()
    while line:
        lines.append(line)
        line = f.readline()
txt = open('jp_full0319.txt', 'w') # 書き込みモードで開く

#count= 0;
texts_set = set()
for i in lines:
    #count +=1
    dic  = json.loads(i)
    s1 = dic['sentence1']
    s2 = dic['sentence2']
    
    #origin data set
    #txt.write(s1+ "\n")
    #txt.write(s2+ "\n")
    
    
    #uniq data set
    if   s1 not in texts_set:
        txt.write(s1+ "\n") 
        texts_set.add(s1)
    if  s2 not in texts_set:
        txt.write(s2+ "\n")
        texts_set.add(s2)
 
#    if(100<count):
#        break
    
txt.close()

FileNotFoundError: [Errno 2] No such file or directory: ''

In [ ]:
#京大含意関係認識用

# input

#kana
lines = []
with open('data/ku_data.txt') as f:
    line = f.readline()
    while line:
        lst = line.split(" ")
        
        #lst[3]= lst[3].replace('\r','')
        #lst[3] = lst[3].replace('。','。\n')
        
        lst[4] = lst[4].replace('\n','')
        lst[4] = lst[4].replace('\r','')
        lst[4] = lst[4].replace('。','。\n')

        lines.append(lst[3]+"\n")
        lines.append(lst[4])        
        line = f.readline()
        
#print(len(lines)/2)

txt = open('test.txt', 'w') # 書き込みモードで開く


count= 0;
for i in lines:
    count +=1
    #print(i)
    txt.write(i) 
    #if(100<count):
    #    break
    
txt.close()


In [3]:
#変数を統一するための処理
#ラムダを取っている


pre_var = []
formulas = []
check_dup = set([])

#from nltk.sem.logic import LogicParser
#from nltk.sem.logic import *

logic_parser = LogicParser(type_check=False)
def lexpr(formula_str):
    return logic_parser.parse(formula_str)

def coq_string_expr(expression):
    
    org_exp = expression
    
    if isinstance(expression, str):
        expression = lexpr(expression)
    expr_coq_str = ''
    if isinstance(expression, ApplicationExpression):
        coq_string_application_expr(expression)
    elif isinstance(expression, AbstractVariableExpression):
        expr_coq_str = coq_string_abstract_variable_expr(expression)
    elif isinstance(expression, LambdaExpression):
        raise
    elif isinstance(expression, QuantifiedExpression):
        expr_coq_str = coq_string_quantified_expr(expression)
    elif isinstance(expression, AndExpression):
        expr_coq_str = coq_string_and_expr(expression)
    elif isinstance(expression, OrExpression):
        expr_coq_str = coq_string_or_expr(expression)
    elif isinstance(expression, NegatedExpression):
        expr_coq_str = coq_string_not_expr(expression)
    elif isinstance(expression, BinaryExpression):
        expr_coq_str = coq_string_binary_expr(expression)
    elif isinstance(expression, Variable):
        expr_coq_str = '%s' % expression
    else:
        expr_coq_str = str(expression)

    return org_exp

coqstr = coq_string_expr

def coq_string_application_expr(expression):
    # uncurry the arguments and find the base function
    if expression.is_atom():
        function, args = expression.uncurry()
        arg_str = ' '.join("%s" % coqstr(arg) for arg in args)
    else:
        #Leave arguments curried
        function = expression.function
        arg_str = "%s" % coqstr(expression.argument)
    function_str = "%s" % coqstr(function)
    parenthesize_function = False
    if isinstance(function, LambdaExpression):
        if isinstance(function.term, ApplicationExpression):
            if not isinstance(function.term.function,
                              AbstractVariableExpression):
                parenthesize_function = True
        elif not isinstance(function.term, BooleanExpression):
            parenthesize_function = True
    elif isinstance(function, ApplicationExpression):
        parenthesize_function = True
    if parenthesize_function:
        function_str = Tokens.OPEN + function_str + Tokens.CLOSE
    return expression 

def coq_string_abstract_variable_expr(expression):
    expr_str = str(expression.variable)
    if not isinstance(expression, FunctionVariableExpression):
        if expr_str == '':
            expr_str = "%s" % expr_str
        else:
            expr_str = "%s" % expr_str
    else:
        expr_str = "%s" % expr_str
        
    if not re.sub(r'_', "",expr_str) in check_dup:
        if not expr_str.startswith('_'):
            pre_var.append(expr_str)
            check_dup.add(expr_str)
    return expression


def coq_string_quantified_expr(expression):
    variables = [expression.variable]
    term = expression.term
    while term.__class__ == expression.__class__:
        variables.append(term.variable)
        term = term.term
    for v in variables:
        coqstr(v)
    coqstr(term) 
    return expression

def coq_string_and_expr(expression):
    first = coqstr(expression.first)
    second = coqstr(expression.second)
    return expression 

def coq_string_or_expr(expression):
    first = coqstr(expression.first)
    second = coqstr(expression.second)
    return expression

def coq_string_not_expr(expression):
    term_str = coqstr(expression.term)
    return expression

def coq_string_binary_expr(expression):
    first = coqstr(expression.first)
    second = coqstr(expression.second)
    return expression


def substituteString(text,lst):
    ###辞書作り###
    lst = list(set(lst))
    e_sub = {}
    z_sub = {}
    ice = 0 #index counter
    icz = 0 

    for i in lst:
        matchObj = re.search(r'e0+', i)
        if(matchObj):
            tmp = "e0"+str(ice)
            e_sub[tmp] = i
            ice+=1
            continue
        matchObj = re.search(r'z[0-9]*', i)
        if matchObj:
            tmp = "z0"+str(icz)
            z_sub[tmp] = i
            icz+=1
    
    e_sub = {v:k for k, v in e_sub.items()}
    z_sub = {v:k for k, v in z_sub.items()}
    
    for k, v in e_sub.items():  
        text = text.replace(k,v)
    for k, v in z_sub.items():  
        text = text.replace(k,v)
    return text



a1 = 'a'
a2 = 'a'

for fname in glob.glob('/Users/guru/Downloads/snli_train_data/parsed/hypothesis/snli-train-h-*.candc.sem.xml'):
    print(a1+a2)
    print (fname)
    
    pre_var = []
    formulas = []
    check_dup = set([])

    tree = ET.parse(fname)#('/Users/guru/Downloads/snli_train_data/parsed/text/snli-train-t-aa.candc.sem.xml')
    root = tree.getroot()

    root = root[0]
    root = root[0] #１個目のsentence

    mydict = {}
    mydict2 = {}

    #c = 0 #辞書のindexを回す
    print(len(root))

    #子階層のタグと中身
    for c,child in enumerate(root):

        formula = child[2]  #child[2]がsemantics
        check = (child[2].attrib)
        check = check['status']


        if(check == 'success'):
            plain = ""
            base = ""
            toridashi = child[0]

            for i in toridashi :
                att = i.attrib
                p = att['surf']
                b = att['base']
                if(p=='.' or p== ','):
                    plain = plain + p
                    base = base + b
                else :
                    plain = plain + " " + p
                    base = base + " " + b
            plain = plain + '\n'
            base = base + '\n'

            formula = child[2][0]
            formula = (formula.attrib)
            formula = formula['sem'] #\nが付与されていると信じている

            #論理式を今回の標準にする
            check_dup = set([])
            pre_var = []

            try:
                org_f = formula
                formula = coq_string_expr(formula)
            except:
                #import traceback
                #traceback.print_exc()
                continue

            if(formula == 'lambda_expression'):
                continue
            formula = substituteString(formula,pre_var)
            ice = 0
            icz = 0

            for i in range(len(pre_var)):
                matchObj = re.search(r'e0+', pre_var[i])
                if(matchObj):
                    tmp = "e0"+str(ice)
                    pre_var[i] = tmp

                matchObj = re.search(r'z[0-9]*', pre_var[i])
                if matchObj:
                    tmp = "z0"+str(icz)
                    pre_var[i] = tmp
                    icz+=1

            pair = {'text':plain,'formula':formula}
            pair2 = {'text':plain,'formula':formula,'base':base}
            mydict.update({str(c):pair})
            mydict2.update({str(c):pair2})

        else:
          print('faild parse')
        #c+=1
        #if(c==100):
        #    break
    print(len(mydict))

    f = open('snli_full/eng_aa.json', 'w') # 書き込みモードで開く
    json.dump(mydict, f,ensure_ascii=False)
    f.close()

    f = open('tmp/h/'+a1+a2+'.txt', 'w') # 書き込みモードで開く
    json.dump(mydict2, f,ensure_ascii=False)
    f.close()
        
    a2 = chr(ord(a2) + 1)
    if(ord(a2)==123):
        a2 = 'a'
        a1 = chr(ord(a1) + 1)


aa
/Users/guru/Downloads/snli_train_data/parsed/hypothesis/snli-train-h-aa.candc.sem.xml
10000
9934
ab
/Users/guru/Downloads/snli_train_data/parsed/hypothesis/snli-train-h-ab.candc.sem.xml
9999
9901
ac
/Users/guru/Downloads/snli_train_data/parsed/hypothesis/snli-train-h-ac.candc.sem.xml
9994
faild parse
9806
ad
/Users/guru/Downloads/snli_train_data/parsed/hypothesis/snli-train-h-ad.candc.sem.xml
9987
faild parse
faild parse
faild parse
9969
ae
/Users/guru/Downloads/snli_train_data/parsed/hypothesis/snli-train-h-ae.candc.sem.xml
9987
faild parse
9455
af
/Users/guru/Downloads/snli_train_data/parsed/hypothesis/snli-train-h-af.candc.sem.xml
9993
faild parse
faild parse
faild parse
9968
ag
/Users/guru/Downloads/snli_train_data/parsed/hypothesis/snli-train-h-ag.candc.sem.xml
9990
faild parse
faild parse
faild parse
9808
ah
/Users/guru/Downloads/snli_train_data/parsed/hypothesis/snli-train-h-ah.candc.sem.xml
9993
faild parse
faild parse
faild parse
9910
ai
/Users/guru/Downloads/snli_train_dat

In [27]:
import glob

a = 'a'
b = 'a'


for fname in glob.glob('/Users/guru/Downloads/snli_train_data/parsed/hypothesis/snli-train-h-*.candc.sem.xml'):
    print(a+b)
    print (fname)
    
    b = chr(ord(b) + 1)
    if(ord(b)==123):
        b = 'a'
        a = chr(ord(a) + 1)


aa
97
/Users/guru/Downloads/snli_train_data/parsed/hypothesis/snli-train-h-aa.candc.sem.xml
ab
98
/Users/guru/Downloads/snli_train_data/parsed/hypothesis/snli-train-h-ab.candc.sem.xml
ac
99
/Users/guru/Downloads/snli_train_data/parsed/hypothesis/snli-train-h-ac.candc.sem.xml
ad
100
/Users/guru/Downloads/snli_train_data/parsed/hypothesis/snli-train-h-ad.candc.sem.xml
ae
101
/Users/guru/Downloads/snli_train_data/parsed/hypothesis/snli-train-h-ae.candc.sem.xml
af
102
/Users/guru/Downloads/snli_train_data/parsed/hypothesis/snli-train-h-af.candc.sem.xml
ag
103
/Users/guru/Downloads/snli_train_data/parsed/hypothesis/snli-train-h-ag.candc.sem.xml
ah
104
/Users/guru/Downloads/snli_train_data/parsed/hypothesis/snli-train-h-ah.candc.sem.xml
ai
105
/Users/guru/Downloads/snli_train_data/parsed/hypothesis/snli-train-h-ai.candc.sem.xml
aj
106
/Users/guru/Downloads/snli_train_data/parsed/hypothesis/snli-train-h-aj.candc.sem.xml
ak
107
/Users/guru/Downloads/snli_train_data/parsed/hypothesis/snli-train